This notebook takes the TrackMate tracks of diffusing foci and looks at their diffusion, correlations between intensity and diffusion constant, and distribution of intensities.

In [ ]:
# load the custom modules
%load_ext autoreload
%autoreload 2

import os, sys, inspect
import matplotlib
import matplotlib.pylab as plt
import numpy as np
import math
from scipy import stats
import random
import pandas as pd
import seaborn as sns

# Add source code directory to path to enable module import
curr_frame = inspect.getfile(inspect.currentframe())
curr_dir = os.path.dirname(os.path.abspath(curr_frame))
parent_dir = os.path.dirname(curr_dir)
module_dir = os.path.join(parent_dir, 'src')
os.sys.path.insert(0, module_dir)

import parse_trackmate as pt
import diffusion as dif

In [ ]:
# Set up plot export and plotting styles

# Plotting and figure saving params
plot_figs = True
save_figs = True
save_dir = '../reports/figures/IRE1_foci_tracking_U2OS/MSD-by-Int_SingleParticle'
fraction_trajectories_to_plot = 0.003 # For generating some representative figs

# create save figure dir and set up figure/font sizes
if save_figs:
    %matplotlib
    
    WIDE_FIG = 1.6, 3.1
    
    matplotlib.rcParams['figure.figsize'] = WIDE_FIG
    save_dir_pdf = os.path.join(save_dir, 'pdf')
    if not os.path.exists(save_dir_pdf):
        os.makedirs(save_dir_pdf)
    
    # Set up fonts
    matplotlib.rc("font", family="Arial")

    matplotlib.rcParams['pdf.fonttype'] = 42 # Make fonts editable
    matplotlib.rcParams['axes.linewidth']= 0.5
    matplotlib.rcParams['lines.linewidth'] = 0.5

    SMALL_SIZE = 5
    MEDIUM_SIZE = 6
    BIGGER_SIZE = 7

    plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
    
else:
    %matplotlib inline
    matplotlib.rcParams['figure.figsize'] = 6, 5.25

In [ ]:
# Load data
import glob

# Path to raw TrackMate data
#data_dir = '../data/processed/IRE1_foci_diffusion_MEFs'
data_dir = '../data/processed/IRE1_foci_Diffusion_U2OS'

data_files = sorted(glob.glob(os.path.join(data_dir,'*.xml')))
stripped_names = [os.path.basename(f) for f in data_files]
file_names = [os.path.splitext(fn)[0] for fn in stripped_names]

# Read data from the provided source files
raw_data = [pt.read_trackmate_file(f) for f in data_files]


In [ ]:
# Parse the data into tracks, creating lists of spots that
# make up each individual track 

# Store all track data
processed_tracks_by_movie = []
spots_by_movie = []

# Process each trace
for file, trace in zip(data_files, raw_data):
    # Parse the data into tracks, creating lists of spots that
    # make up each individual track 
    raw_tracks = pt.get_raw_tracks(trace)
    spots = pt.list_spots(trace)
    processed_tracks = pt.build_spot_lists_by_track(raw_tracks, spots)
    track_coords, track_edges, track_spot_ids = processed_tracks
    
    # Get intensity values for each frame of each track
    all_track_intensities = pt.pull_property_by_track('TOTAL_INTENSITY',
                                                   track_spot_ids, spots)
   
    # Check that spot radii are constant (otherwise comparison of 
    # mean intensities is meaningless)
    all_spot_radii = pt.pull_property_by_track('RADIUS',
                                                track_spot_ids, spots)
    radii_norm = np.concatenate(all_spot_radii - all_spot_radii[0][0])
    diff_radii = np.count_nonzero(radii_norm)
    if diff_radii > 0: print('WARNING: spot sizes are not uniform')

    processed_tracks_by_movie.append(processed_tracks)
    spots_by_movie.append(spots)



In [ ]:
# Clean up garbage (the dom files can take up lots of RAM)
import gc

del raw_tracks
del raw_data
gc.collect()

### All data are now loaded. Analysis cells follow.

In [ ]:
# Analyze MSD vs. intensity for single particles

# Input imaging parameters:
frame_interval = 1 # in seconds

# Fitting parameters
diff_dim = 2 # 1D, 2D, or 3D diffusion
min_averages_for_msd = 3 # minimum number of trajectories to be averaged for MSD calculation
dc_fit_nframes = 10 # number of frames to fit to get diffusion constant
    
# Make save dir
dir_examples = os.path.join(save_dir, 'examples') 
if save_figs and not os.path.exists(save_dir):
    os.makedirs(save_dir)
if save_figs and not os.path.exists(dir_examples):
    os.makedirs(dir_examples)

random.seed(30)
    
if plot_figs:
    %matplotlib
    
# Pool all tracks together
all_track_c = [] # coordinates (list of x,y,t numpy arrays)
all_track_int = [] # intensities

for curr_tracks, spots in zip(processed_tracks_by_movie, spots_by_movie):
    track_coords, track_edges, track_spot_ids = curr_tracks
    
    # Get intensity values for each frame of each track
    track_intensities = pt.pull_property_by_track('TOTAL_INTENSITY',
                                                   track_spot_ids, spots)

    all_track_c = all_track_c + track_coords
    all_track_int = all_track_int + track_intensities


d = []

# Individually calculate MSD for each track and fit to diffusion constant
for i, (track, intensity) in enumerate(zip(all_track_c, all_track_int)):
    len_track = np.shape(track)[0]
    if len_track < (dc_fit_nframes*min_averages_for_msd):
        continue
    start_int = intensity[0]
    mean_int = np.mean(intensity)
    
    # Break up the track into sub-trajectories for MSD calulation
    track_chunks = []
    n_chunks = int(np.floor(len_track / dc_fit_nframes))
    for i in range(n_chunks):
        start_slice = i*dc_fit_nframes
        end_slice = start_slice + dc_fit_nframes
        chunk = track[start_slice:end_slice, :]
        track_chunks.append(chunk)
    
    #t_dsq, msd_data = dif.calc_msd([track], frame_interval)
    t_dsq, msd_data = dif.calc_msd(track_chunks, frame_interval)
    
    time, mean_dsq, std_dsq, sterr_dsq = msd_data
    
    
    fit_params = dif.fit_diffusion_const(msd_data, dim=diff_dim, 
                                         nframes=dc_fit_nframes)
    
    # save a few randomly chosen trajectory plots
    if (plot_figs or save_figs) and random.random() < fraction_trajectories_to_plot:
        fig, axarr = dif.plot_msd(t_dsq, msd_data, fit_params, rc_params=[3.5,2])
        fig.tight_layout(pad=2)
        if save_figs:
            filename = 'Track ID_'+str(i)+'_Mean_Intensity_'+str(mean_int)
            print(filename)
            dif.save_fig_pdf(fig, filename, dir_examples)
    
    d.append({'Track_ID':i, 'Start_Int':start_int, 'Mean_Int':mean_int,
              'Diff_const':fit_params['dc']})

gc.collect()

data = pd.DataFrame(d)

data['Sqrt_Int'] = np.sqrt(data['Mean_Int'])


# Plot summaries
if plot_figs or save_figs:
    
    fig, ax = plt.subplots() 
    sns.regplot(x='Mean_Int', y='Diff_const', data=data, ax=ax, x_bins=20, ci=98,
               scatter_kws={'s':3})
    
    # This triggers a Seaborn future warning error
    slope, intercept, r_value, p_value, std_err = stats.linregress(data['Mean_Int'], 
                                                                   data['Diff_const'])

    print(slope, intercept, r_value, p_value, std_err)
    
    ax.set_ylim(0,)
    ax.set_xlim(0,)
    fig.tight_layout(pad=2)
    
    if save_figs:
        filename = 'MSD-by-Int_SingleParticle'
        dif.save_fig_pdf(fig, filename, save_dir)

    plt.show()
else:
    plt.close('all')
    print('done!')


In [ ]:
# Plot trajectories
%matplotlib

# Make save dir
dir_traj = os.path.join(save_dir, 'trajectories') 
if save_figs and not os.path.exists(dir_traj):
    os.makedirs(dir_traj)

    
for curr_tracks, spots, file_name in zip(processed_tracks_by_movie, 
                              spots_by_movie, stripped_names):
    track_coords, track_edges, track_spot_ids = curr_tracks
    track_intensities = pt.pull_property_by_track('TOTAL_INTENSITY',
                                                   track_spot_ids, spots)
    
    # Various plotting tools
    fig, axarr = plt.subplots(1,2)
    
    fig.tight_layout(pad=2)
    
    axarr[0].set_title('All tracks')
    axarr[0].set_xlabel('Position (μm)')
    axarr[0].set_ylabel('Position (μm)')
    axarr[1].set_xlabel('Frame')
    axarr[1].set_ylabel('Intensity (A.U.)')
    axarr[1].set_title('All intensities')
    for i, track in enumerate(track_coords):
        axarr[0].plot(track[:,0], track[:,1])
        axarr[1].plot(track[:,2], track_intensities[i])
        
    if save_figs:
        title = file_name + '_all_trajectories'
        dif.save_fig_pdf(fig, title, dir_traj)
        
    plt.show()


In [ ]:
# This cell estimates cluster diffusion parameters based on the Guigas and Weiss 2006 paper, 
# doi 10.1529/biophysj.106.087031
if not save_figs:
    %matplotlib inline

T = 310 # 37C in Kelvin
c = 6 # in nm; 4 for slip, 6 for stick boundary conditions
eta = 0.056 # Pa s; taken from paper

ire1_radius = 2.5 # estimated in-plane radius of IRE1 in nm
min_clust = 1000 # smallest number of IRE1 molecules in cluster
max_clust = 10000 # largest number of IRE1 molecules in cluster

ire1_area = ire1_radius**2
clust_sizes = np.arange(min_clust, max_clust, float(max_clust-min_clust) / 100)
clust_r = np.sqrt(clust_sizes * ire1_area / np.pi)


D = np.array([dif.diffconst_gw(r,T,c,eta) for r in clust_r])

# Uncomment to replicate original paper plot:
"""
R = np.arange(1,100,0.1) 
D = np.array([dif.diffconst_gw(r,T,c) for r in R])
D_sd = np.array([dif.diffconst_sd(r,T,c) for r in R])
plt.loglog(R, D)
plt.loglog(R, D_sd)
"""

# Plot results
plt.loglog(clust_sizes, D)
plt.show()

In [ ]:
# Plot comparison of measured diffusion constants vs. model based on
# the Guigas and Weiss 2006 paper, doi 10.1529/biophysj.106.087031

# Model params
T = 310 # 37C in Kelvin
c = 6 # in nm; 4 for slip, 6 for stick boundary conditions
eta = 0.056 # Pa s; taken from paper

ire1_radius = 2.5 # estimated in-plane radius of IRE1 in nm
min_clust = 200 # smallest number of IRE1 molecules in cluster
max_clust = 2000 # largest number of IRE1 molecules in cluster

ire1_area = ire1_radius**2
clust_sizes = np.arange(min_clust, max_clust, float(max_clust-min_clust) / 100)
clust_r = np.sqrt(clust_sizes * ire1_area / np.pi)
D = np.array([dif.diffconst_gw(r,T,c,eta) for r in clust_r])

# rescale model intensities based on smallest and largest measured clusters,
# assuming intensities scale linearly with cluster area
min_int = min(data['Mean_Int'])

a = min_int / min_clust
model_ints = clust_sizes * a

if plot_figs or save_figs:
    fig, ax = plt.subplots() 
    sns.regplot(x='Mean_Int', y='Diff_const', data=data, ax=ax, x_bins=20, ci=95,
               scatter_kws={'s':3})
    ax.plot(model_ints, D)
    ax.loglog()
    
    # This triggers a Seaborn future warning error
    slope, intercept, r_value, p_value, std_err = stats.linregress(data['Mean_Int'], 
                                                                   data['Diff_const'])

    print(slope, intercept, r_value, p_value, std_err)

    fig.tight_layout(pad=2)
    
    if save_figs:
        filename = 'MSD-by-Int_SingleParticle_vs_model'
        dif.save_fig_pdf(fig, filename, save_dir)

    plt.show()

In [ ]:
# Fit data to D ~ const, D ~ 1/R (incompressible inclusion), and D ~ 1/R^2 (from
# the Guigas and Weiss 2006 paper, doi 10.1529/biophysj.106.087031) and
# estimate  confidence intervals by bootstrapping.

#normalize intensities
data['Mean_Int_Norm'] = data['Mean_Int'] / max(data['Mean_Int'])

# Convert data to log-log for linear regression fitting
log_int = np.log10(data['Mean_Int_Norm']).copy()
log_D = np.log10(data['Diff_const']).copy()
log_int.name = "Log10_Intensities"
log_D.name = "Log10_Diffusion_Constant"

# Fit data to three different models
fit_linreg = dif.bootstrap_linreg(log_int, log_D, slope=None)
fit_1R = dif.bootstrap_linreg(log_int, log_D, slope=-1)
fit_2R = dif.bootstrap_linreg(log_int, log_D, slope=-2)

print(fit_linreg)
print(fit_1R)
print(fit_2R)

#plot results
if plot_figs or save_figs:
    
    #  generate lines for fixed-slope fits
    xmin = min(log_int)
    xmax = max(log_int)
    int_lin_x = np.arange(xmin*0.99, xmax*1.01, float(xmax-xmin)/20)
    y_1_R = fit_1R['slope'] * int_lin_x + fit_1R['y_int']
    y_2_R = fit_2R['slope'] * int_lin_x + fit_2R['y_int']
    y_regression = fit_linreg['slope'] * int_lin_x + fit_linreg['y_int']
      
    fig, ax = plt.subplots() 
    
    sns.scatterplot(x=10**log_int, y=10**log_D, ax=ax, 
                s=3, color='gray', linewidth=0)
    
    ax.plot(10**int_lin_x, 10**y_regression)
    ax.plot(10**int_lin_x, 10**y_1_R)
    ax.plot(10**int_lin_x, 10**y_2_R)
    ax.loglog()
    fig.tight_layout(pad=2)
 
    fig2, ax2 = plt.subplots(1,3, sharey=True) 
    
    sns.scatterplot(x=10**log_int, y=10**dif.get_residuals(log_int,log_D,fit_linreg['slope'],
                                                           fit_linreg['y_int']),
                    ax=ax2[0], s=3, linewidth=0)
    ax2[0].loglog()
    sns.scatterplot(x=10**log_int, y=10**dif.get_residuals(log_int,log_D,-1,fit_1R['y_int']),
                    ax=ax2[1], s=3, linewidth=0)
    ax2[1].loglog()
    sns.scatterplot(x=10**log_int, y=10**dif.get_residuals(log_int,log_D,-2,fit_2R['y_int']),
                    ax=ax2[2], s=3, linewidth=0)
    ax2[2].loglog()  
    ax2[0].set_ylabel('Log10 diffusion constant residuals')
    
    if save_figs:
        filename = 'Different_power_MSD_fits'
        filename_residuals = 'Residual_MSD_plots_regression_1R_2R'
        dif.save_fig_pdf(fig, filename, save_dir)
        dif.save_fig_pdf(fig2, filename_residuals, save_dir)

    plt.show()



In [ ]:
print(len(all_track_c))